# Walmart - Pharmacy Consultation Privacy for Patient Comfort

```SQL
CREATE TABLE fct_consultations (
    consultation_id INTEGER,
    pharmacy_name VARCHAR,
    consultation_date DATE,
    consultation_room_type VARCHAR,
    privacy_level_score INTEGER
);

INSERT INTO fct_consultations (
    consultation_id,
    pharmacy_name,
    consultation_date,
    consultation_room_type,
    privacy_level_score
)
VALUES
    (1, 'Walmart Pharmacy North', '2024-07-05', 'private', 8),
    (2, 'Walmart Pharmacy South', '2024-07-07', 'semi-private', 6),
    (3, 'Walmart Pharmacy East', '2024-07-10', 'open', 5),
    (4, 'Walmart Pharmacy West', '2024-07-12', 'private', 9),
    (5, 'Walmart Pharmacy Central', '2024-07-14', 'group', 7),
    (6, 'Walmart Pharmacy North', '2024-07-20', 'open', 6),
    (7, 'Walmart Pharmacy South', '2024-07-21', 'private', 8),
    (8, 'Walmart Pharmacy East', '2024-07-22', 'semi-private', 7),
    (9, 'Walmart Pharmacy West', '2024-07-23', 'group', 4),
    (10, 'Walmart Pharmacy Central', '2024-07-24', 'soundproof', 9),
    (11, 'Walmart Pharmacy Central', '2024-07-25', 'private', 8),
    (12, 'Walmart Pharmacy North', '2024-07-29', 'semi-private', 7),
    (13, 'Walmart Pharmacy South', '2024-08-05', 'open', 5),
    (14, 'Walmart Pharmacy East', '2024-08-06', 'soundproof', 10),
    (15, 'Walmart Pharmacy West', '2024-08-07', 'semi-private', 7);
```

In [ ]:
import pandas as pd
import numpy as np

In [19]:
df_consultation = pd.read_csv('Data/013/fct_consultations_2.csv')
df_consultation.head()

,consultation_id,pharmacy_name,consultation_date,consultation_room_type,privacy_level_score
0,1,Walmart Pharmacy North,2024-07-05,private,8
1,2,Walmart Pharmacy South,2024-07-07,semi-private,6
2,3,Walmart Pharmacy East,2024-07-10,open,5
3,4,Walmart Pharmacy West,2024-07-12,private,9
4,5,Walmart Pharmacy Central,2024-07-14,group,7


# Preguntas 1

### ¿Cuáles son los nombres de las 3 farmacias que realizaron el menor número de consultas en julio de 2024? Esto nos ayudará a identificar las ubicaciones con espacios de consulta potencialmente menos concurridos.

In [ ]:
print(df_consultation.columns)

In [20]:
# 1. Convertimos la columna a fecha (Correcto)
df_consultation['consultation_date'] = pd.to_datetime(df_consultation['consultation_date'])

# 2. Filtramos julio 2024 (Correcto)
df_julio = df_consultation[
    (df_consultation['consultation_date'].dt.year == 2024) &
    (df_consultation['consultation_date'].dt.month == 7)
]

# 3. Agrupamos y contamos (Corregido 'groupby')
respuesta = df_julio.groupby('pharmacy_name', as_index=False)['consultation_id'].count().sort_values(by='consultation_id',ascending=True).head(3)

respuesta

,pharmacy_name,consultation_id
1,Walmart Pharmacy East,2
3,Walmart Pharmacy South,2
4,Walmart Pharmacy West,2


```SQL
SELECT
    pharmacy_name,
    COUNT(consultation_id)
FROM fct_consultations
WHERE EXTRACT (YEAR FROM consultation_date) = 2024
AND EXTRACT (MONTH FROM consultation_date) = 7
GROUP BY pharmacy_name
ORDER BY count(consultation_id) ASC
LIMIT 3;
```

# Pregunta 2

### Para las farmacias identificadas en la pregunta anterior (es decir, las 3 farmacias con el menor número de consultas en julio de 2024), ¿cuál es la versión en mayúsculas de los tipos de sala de consulta disponibles? Comprender los tipos de sala puede proporcionar información sobre las características de privacidad ofrecidas.

In [14]:
# 1. Filtramos el DataFrame original usando el índice de tu 'respuesta'
df_top_3 = df_consultation[df_consultation['pharmacy_name'].isin(respuesta['pharmacy_name'])]

# 2. Obtenemos los tipos únicos, los pasamos a mayúsculas y listo
tipos_mayuscula = df_top_3['consultation_room_type'].str.upper().unique()

print(tipos_mayuscula)

['SEMI-PRIVATE' 'OPEN' 'PRIVATE' 'GROUP' 'SOUNDPROOF']


```SQL
SELECT
    DISTINCT UPPER(consultation_room_type)
FROM fct_consultations
WHERE pharmacy_name IN (
    SELECT
        pharmacy_name
    FROM fct_consultations
    WHERE EXTRACT(YEAR FROM consultation_date) = 2024
      AND EXTRACT(MONTH FROM consultation_date) = 7
    GROUP BY pharmacy_name
    ORDER BY COUNT(consultation_id) ASC
    LIMIT 3
);
```

# Pregunta 3

### Hasta ahora, hemos identificado las 3 farmacias con menos consultas en julio de 2024. Entre estas 3 farmacias, ¿cuál es el puntaje de nivel de privacidad mínimo para cada tipo de sala de consulta en julio de 2024?

In [26]:
df_top_3_julio = df_julio[df_julio['pharmacy_name'].isin(respuesta['pharmacy_name'])]

resultado_minimos = df_top_3_julio.groupby('consultation_room_type', as_index=False)['privacy_level_score'].min()
    
resultado_minimos

,consultation_room_type,privacy_level_score
0,group,4
1,open,5
2,private,8
3,semi-private,6


```SQL
SELECT
    consultation_room_type,
    MIN(privacy_level_score) AS min_privacy_level
FROM fct_consultations
WHERE pharmacy_name IN (
    SELECT
        pharmacy_name
    FROM fct_consultations
    WHERE EXTRACT(YEAR FROM consultation_date) = 2024
      AND EXTRACT(MONTH FROM consultation_date) = 7
    GROUP BY pharmacy_name
    ORDER BY COUNT(consultation_id) ASC
    LIMIT 3
)
AND EXTRACT (YEAR FROM consultation_date) = 2024
AND EXTRACT (MONTH FROM consultation_date) = 7
GROUP BY consultation_room_type
ORDER BY min_privacy_level ASC;
```